# 0.0 IMPORT

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

## 0.1 Coleta de dados

In [2]:
path = '/home/borges/'
database_name = 'database_hm.sqlite'
conn = create_engine('sqlite:///' + path + database_name, echo=False)

In [3]:
query = """
    SELECT * FROM vitrine
"""

In [4]:
df_raw = pd.read_sql(query, con=conn)

In [5]:
df_raw.sample(8)

,product_id,style_id,color_id,product_name,color_name,fit,product_pric,size_number,size_model,cotton,polyester,elastane,elasterell,scrapy_datetime
217,0730863033,0730863,033,skinny_jeans,black/no_fade_black,skinny_fit,29.99,None,None,0.98,0.00,0.02,0.0,2021-09-06 19:41:22
193,0690449051,0690449,051,skinny_jeans,gray,skinny_fit,39.99,None,None,0.98,0.00,0.02,0.0,2021-09-06 19:41:22
788,1008549006,1008549,006,regular_jeans,black,regular_fit,19.99,None,None,1.00,0.00,0.01,0.0,2021-09-07 13:59:24
438,0636207011,0636207,011,slim_jeans,midnight_blue,slim_fit,19.99,None,None,0.89,0.65,0.01,0.0,2021-09-07 10:23:00
486,1013317001,1013317,001,hybrid_regular_tapered_joggers,light_denim_blue,regular_fit,39.99,None,None,0.77,0.21,0.02,0.0,2021-09-07 10:23:00
234,0730863040,0730863,040,skinny_jeans,denim_blue,skinny_fit,29.99,None,None,0.98,0.00,0.02,0.0,2021-09-06 19:41:22
1037,0720504004,0720504,004,skinny_jeans,light_denim_blue,skinny_fit,12.99,None,None,1.00,0.14,0.01,0.0,2021-09-08 09:52:51
108,0811993025,0811993,025,regular_jeans,light_blue,regular_fit,15.99,None,None,1.00,0.00,0.01,0.0,2021-09-06 19:25:43


# 1.0 Passo 01 - Descrição dos dados

In [6]:
df01 = df_raw.copy()

## 1.1 Dimesão dos dados

In [7]:
print('Number of Rows: {}'.format(df01.shape[0]))
print('Number of Columns: {}'.format(df01.shape[1]))

Number of Rows: 1168
Number of Columns: 14


## 1.2 Tipos de dados

In [8]:
df01.dtypes

product_id          object
style_id            object
color_id            object
product_name        object
color_name          object
fit                 object
product_pric       float64
size_number         object
size_model          object
cotton             float64
polyester          float64
elastane           float64
elasterell         float64
scrapy_datetime     object
dtype: object

In [9]:
# convert object to datetime
df01['scrapy_datetime'] = pd.to_datetime(df01['scrapy_datetime'])
df01.dtypes

product_id                 object
style_id                   object
color_id                   object
product_name               object
color_name                 object
fit                        object
product_pric              float64
size_number                object
size_model                 object
cotton                    float64
polyester                 float64
elastane                  float64
elasterell                float64
scrapy_datetime    datetime64[ns]
dtype: object

## 1.3 Idetificação de dados faltantes

In [10]:
df01.isna().sum()

product_id           0
style_id             0
color_id             0
product_name         0
color_name           0
fit                  0
product_pric         0
size_number        918
size_model         934
cotton               0
polyester            0
elastane             0
elasterell           0
scrapy_datetime      0
dtype: int64

In [11]:
df01.isna().sum() / df01.shape[0]

product_id         0.000000
style_id           0.000000
color_id           0.000000
product_name       0.000000
color_name         0.000000
fit                0.000000
product_pric       0.000000
size_number        0.785959
size_model         0.799658
cotton             0.000000
polyester          0.000000
elastane           0.000000
elasterell         0.000000
scrapy_datetime    0.000000
dtype: float64

## 1.4 Substituição dos dados faltantes

In [12]:
df01 = df01.drop(columns=['size_number', 'size_model']).dropna()

### 1.5.1 Numeric data

## 1.5 Descrição dos dados

In [13]:
num_attributes = df01.select_dtypes(include=['int64', 'float64'])
cat_attributes = df01.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.5.2 Categorical data

In [14]:
# tendencia central - média, mediana
t1 = pd.DataFrame(num_attributes.apply(np.mean)).T
t2 = pd.DataFrame(num_attributes.apply(np.median)).T

# disperson - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concat
m1 = pd.concat([d2, d3, d4, t1, t2, d1, d5, d6]).T.reset_index()
m1.columns =['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'shew', 'kurtosis']
m1

,attributes,min,max,range,mean,median,std,shew,kurtosis
0,product_pric,5.99,49.99,44.00,25.551644,21.990,10.471077,0.733884,-0.052290
1,cotton,0.66,1.00,0.34,0.961849,0.980,0.059080,-2.659309,7.876482
2,polyester,0.00,1.00,1.00,0.138699,0.000,0.266045,1.938449,2.481455
3,elastane,0.00,0.02,0.02,0.014521,0.015,0.005862,-0.528434,-0.650466
4,elasterell,0.00,0.08,0.08,0.007671,0.000,0.023555,2.748459,5.563553


# 2.0 Passo 02 - Feature Engineering